In [162]:
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
import json
import numpy
from IPython.display import display
from subprocess import check_output
import sys
import traceback
from tqdm.notebook import tqdm, trange

In [163]:
techs = {
    "tech1": "1q_1o",
    "tech2": "k_per_terra",
    "tech3": "4pip",
    "tech4": "7vp",
    "tech5": "1o_1pw",
    "tech6": "1k_1c",
    "tech7": "3vp_per_gaia_place",
    "tech8": "4c",
    "tech9": "4pw",
    # idk wtf this is --> "tech-ship0": "2c_per_trade"
}

adv_techs = {
    "advtech1": "3vp_per_fed_pass",
    "advtech2": "2vp_per_tech_bump",
    "advtech3": "1qic_5c_action",
    "advtech4": "2vp_per_mine",
    "advtech5": "3vp_per_rl_pass",
    "advtech6": "1o_per_sector",
    "advtech7": "1vp_per_terra_pass",
    "advtech8": "2vp_per_gaia",
    "advtech9": "4vp_per_ts",
    "advtech10": "2vp_per_sector",
    "advtech11": "3o_action",
    "advtech12": "5vp_per_fed",
    "advtech13": "3k_action",
    "advtech14": "3vp_per_mine_place",
    "advtech15": "3vp_per_ts_place",
}

feds = {
    "fed1": "12vp",
    "fed2": "qic",
    "fed3": "2pw",
    "fed4": "2o",
    "fed5": "6c",
    "fed6": "2k",
    "gleens": "gleens"
}

round_scorings = {
    "score1": "2vp_per_terra",
    "score2": "2vp_per_research_bump",
    "score3": "2vp_per_mine_place",
    "score4": "5vp_per_fed_place",
    "score5": "4vp_per_ts_place",
    "score6": "4vp_per_gaia_place",
    "score7": "5vp_per_3pip_place",
    "score8": "3vp_per_ts_place",
    "score9": "3vp_per_gaia_place",
    "score10": "5vp_per_3pip_place",
}

boosters = {

    "booster1": "1k_1o",
    "booster2": "2pwt_1o",
    "booster3": "1qic_2c",
    "booster4": "2c_terra",
    "booster5": "2pw_nav",
    "booster6": "1o_1vp_per_mine",
    "booster7": "1o_2vp_per_ts",
    "booster8": "1k_3vp_per_rl",
    "booster9": "4pw_4vp_per_3pip",
    "booster10": "4c_1vp_per_gaia",
}

bad_buildings = {"colony", 
                 "colonyShip", 
                 "tradeShip", 
                 "constructionShip", 
                 "researchShip",
                 "scout",
                 "frigate",
                 "battleShip",
                 "customsPost",
                 "tradePost"
                }

In [164]:
# for progressbar. Lines are very long jsons, so bash wc way better than native iteration
def wc(filename):
    return int(check_output(["wc", "-l", filename]).split()[0])

In [165]:
total_lines = wc('game_data_raw.txt')
total_lines

64156

In [166]:
def parse_tree_moves(dat, tree, num_players):
    
    tech_slots = {
        'terra',
        'nav',
        'int',
        'gaia',
        'eco',
        'sci'
    }

    # keeping rounds and final scorings seperate for better analysis
    # starting to reach mem limits on server
    point_types = {
        'init',
        'bid',
        'charge',
        'booster',
        'round1',
        'round2',
        'round3',
        'round4',
        'round5',
        'round6',
        'fed',
        'adv',
        'tech_basic',
        'qic',
        'gaia',
        'gleens',
        'final1',
        'final2',
        'resource'
    }
    
    factions_in = []
    factions_rankings = []
    techs_in = {}
    faction_points = dict() # dict of dicts
    
    # get factions in
    for pos in range(num_players):
        faction = tree['players'][pos]['faction']
        factions_in.append(faction)        
        points = dict()
        for point in point_types:
            points[point] = 0
        points['init'] = 10
        faction_points[faction] = points
        factions_rankings.append(tree['players'][pos]['ranking'])
        
    """
    # get dict of tech slots to name
    found_techs = tree['data']['tiles']['techs']
    for slot in found_techs.keys():
        tech_jname = found_techs[slot]
        if tech_jname in techs:
            techs_in[slot] = techs[tech_jname]
            
    """
    moves =  tree['data']['advancedLog']
    move_text = tree['data']['moveHistory']
    
    for move in moves:
        if 'changes' in move.keys():
            changes = move['changes']
            faction = factions_in[move['player']]
            
            # charge
            if 'charge' in changes.keys() and 'vp' in changes['charge'].keys():
                faction_points[faction]['charge'] += changes['charge']['vp']
            
            # booster scoring
            for i in range(10):
                b = 'booster' + str(i + 1)
                if b in changes.keys() and 'vp' in changes[b].keys():
                    faction_points[faction]['booster'] += changes[b]['vp']
                    break
            
            # round
            for i in range(6):
                r = 'round' + str(i + 1)
                if r in changes.keys() and 'vp' in changes[r].keys():
                    faction_points[faction][r] += changes[r]['vp']
                    break
                    
            # qic
            for i in range(3):
                q = 'qic' + str(i + 1)
                if q in changes.keys() and 'vp' in changes[q].keys():
                    faction_points[faction]['qic'] += changes[q]['vp']
                    break
            
            # feds
            if 'federation' in changes.keys() and 'vp' in changes['federation'].keys():
                faction_points[faction]['fed'] += changes['federation']['vp']
                
            # techs
            for slot in tech_slots:
                t = 'tech-' + slot
                if t in changes.keys() and 'vp' in changes[t].keys():
                    faction_points[faction]['tech_basic'] += changes[t]['vp']
                    
                t = 'adv-' + slot
                if t in changes.keys() and 'vp' in changes[t].keys():
                    faction_points[faction]['adv'] += changes[t]['vp']
                    
            for i in range(3):
                t = 'tech-free' + str(i + 1)
                if t in changes.keys() and 'vp' in changes[t].keys():
                    faction_points[faction]['tech_basic'] += changes[t]['vp']
                    break   
            
            # gaia 
            # check that final scoring gaia (tech score) doesn't conflict with reaching top of track
            #     as they have the same key
            if 'gaia' in changes.keys() and 'vp' in changes['gaia'].keys() and 'move' in move.keys():
                faction_points[faction]['gaia'] += changes['gaia']['vp']
                
            # gleens
            if 'gleens' in changes.keys() and 'vp' in changes['gleens'].keys():
                faction_points[faction]['gleens'] += changes['gleens']['vp']
                
            # final scoring 
            if 'final1' in changes.keys() and 'vp' in changes['final1'].keys():
                faction_points[faction]['final1'] += changes['final1']['vp']
            if 'final2' in changes.keys() and 'vp' in changes['final2'].keys():
                faction_points[faction]['final2'] += changes['final2']['vp']
            
            # spend
            if 'spend' in changes.keys() and 'vp' in changes['spend'].keys():
                faction_points[faction]['resource'] += changes['spend']['vp']
            
            # bid
            if 'bid' in changes.keys() and 'vp' in changes['bid'].keys():
                faction_points[faction]['bid'] += changes['bid']['vp']
                
    # tech score already in normal parsing
    
    
    for player in range(num_players):
        faction = factions_in[player]
        rank = factions_rankings[player]
        prefix = 'pos_' + str(rank) + '_score_'
        
        for pt in faction_points[faction].keys():
            dat[prefix + pt] = faction_points[faction][pt]
            
    return dat
        

In [167]:
def parse_tree_builds(dat, tree, pos, faction):
    
    moves = tree['data']['moveHistory']
    logs = tree['data']['advancedLog']
    
    cur_round = 0
    built = {
        'm': 0,
        'ts': 0,
        'lab': 0,
        'ac1': 0,
        'ac2': 0,
        'PI': 0,
        'gf': 0,
    }
    for log in logs:
        if 'round' in log.keys():
            cur_round = log['round']
            if cur_round > 1:
                for key in built.keys():
                    
                    #catch all negative amounts
                    if built[key] < 0:
                        raise ValueError('negative structure amount found : ' + key + ' ' + faction + ' ' + str(cur_round)) 
                    
                    dat[pos + 'buildings_r_' + str(cur_round - 1) + '_' + key] = built[key]
                
        if 'move' in log.keys():
            move = moves[log['move']]
            if 'build' in move and faction in move:
                struct = move.split('build')[1].split()[0]
                if struct in built.keys():
                    built[struct] += 1 
                
                if struct == 'ts' and 'special' not in move:
                    built['m'] -= 1
                elif struct == 'ts' and 'special' in move:
                    built['lab'] -= 1
                elif struct == 'PI' and not faction == 'ivits':
                    if faction == 'bescods':
                        built['lab'] -= 1
                    else:
                        built['ts'] -= 1
                elif struct == 'lab':
                    built['ts'] -= 1
                elif struct == 'ac1' or struct == 'ac2':
                    if faction == 'bescods':
                        built['ts'] -= 1
                    else:
                        built['lab'] -= 1
                
    return dat

In [180]:
"""
Parses non-expansion game data

ToDo: iterate through game moves to get first turn builings, and score vp disterbution

ToDo: add balanced vs unbalanced flag
"""
def parse_tree(tree, errors_set):
    
    if not tree['cancelled'] and tree['status'] == 'ended':
        try:
            dat = {}
            dat['id'] = tree['_id']
            
            # check for expansions
            if 'expansions' in tree['game'].keys() and len(tree['game']['expansions']) > 0:
                raise ValueError('this game is with expansions: ' + ''.join(tree['game']['expansions']))
            if 'expansions' in tree['data'].keys() and tree['data']['expansions'] != 0:
                raise ValueError('this game is with expansions: ' + str(tree['data']['expansions']))

            # some jsons dont have layout. must be before the site supported that feature
            # assume standard
            if 'options' in tree['data'].keys() and 'layout' in tree['data']['options'].keys():
                dat['map_layout'] = tree['data']['options']['layout']
            else:
                dat['map_layout'] = 'standard'
                
            # balanced factions?
            if 'options' in tree['game'].keys():
                if 'factionVariant' in tree['game']['options'].keys():
                    dat['balance_variant'] = tree['game']['options']['factionVariant']
                else:
                    dat['balance_variant'] = 'standard'

            num_players = tree['options']['setup']['nbPlayers']
            dat['num_players'] = num_players
            tot_elo = 0

            #boosters in game
            found_boosters = tree['data']['tiles']['boosters']
            for i in range(10):
                booster_name = 'booster' + str(i + 1)
                if booster_name in found_boosters.keys():
                    dat[boosters[booster_name]] = True
                else:
                    dat[boosters[booster_name]] = False

            #tech locations
            found_techs = tree['data']['tiles']['techs']
            for loc in found_techs.keys():
                name = found_techs[loc]['tile']
                if name in techs:
                    dat['tech_' + loc] = techs[name]
                elif name in adv_techs:
                    dat['tech_' + loc] = adv_techs[name]
                else:
                    raise NameError('unknown tech: ' + name)
                    # dat['tech_' + loc] = name


            #scorings
            found_scorings = tree['data']['tiles']['scorings']['round']
            for i in range(len(found_scorings)):
                dat['round_' + str(i+1) +'_scoring'] = round_scorings[found_scorings[i]]
            dat['final_scoring_1'] = tree['data']['tiles']['scorings']['final'][0]
            dat['final_scoring_2'] = tree['data']['tiles']['scorings']['final'][1]


            #player data
            for i in range(num_players):
                
                #position
                pos = "pos_" + str(tree['players'][i]['ranking']) + "_"
                
                #dropped or no
                dat[pos + 'dropped'] = tree['players'][i]['dropped']

                #elo
                elo = tree['players'][i]['elo']['initial']
                tot_elo += elo
                dat[pos + 'elo'] = elo

                #faction
                faction = tree['players'][i]['faction']
                dat[pos + 'faction'] = faction

                #score
                dat[pos + 'score'] = tree['players'][i]['score']

                #start pos
                dat[pos + 'start_pos'] = [i + 1 for i in range(len(tree['data']['setup'])) if tree['data']['setup'][i] == faction][0]

                #bid
                dat[pos + 'bid'] = tree['data']['players'][i]['data']['bid']

                #feds
                feds_taken = tree['data']['players'][i]['data']['tiles']['federations']
                dat[pos + 'feds_taken'] = len(feds_taken)
                for key in feds.keys():
                    dat[pos + 'fed_' + feds[key]] = 0
                for fed in feds_taken:
                    dat[pos + 'fed_' + feds[fed['tile']]] += 1

                #final buildings
                total_buildings = 0
                found_buildings = tree['data']['players'][i]['data']['buildings']
                for key in found_buildings.keys():
                    if key in bad_buildings:
                        if found_buildings[key] > 0:
                            raise ValueError('this is a game with expansions ' + key)
                        continue
                    elif key != 'gf' and key != 'sp':
                        dat[pos + 'build_' + key] = found_buildings[key]
                        total_buildings += found_buildings[key]
                dat[pos + 'num_structures'] = total_buildings

                #research
                tech_score = 0
                found_research = tree['data']['players'][i]['data']['research']
                for key in found_research.keys():
                    if key == 'dip':
                        if found_research[key] > 0:
                            # normal games seem to have this key now also
                            raise ValueError('this is a game with expansions ' + key)
                        
                    dat[pos + 'research_level_' + key] = found_research[key]
                    if found_research[key] > 2:
                        tech_score += (4 * (found_research[key] - 2))
                dat[pos + 'tech_score'] = tech_score

                #techs taken
                total_techs = 0
                found_techs = tree['data']['players'][i]['data']['tiles']['techs']
                for key in techs.keys():
                    dat[pos + 'tech_taken_' + techs[key]] = False
                for key in adv_techs.keys():
                    dat[pos + 'adv_tech_taken_' + adv_techs[key]] = False
                for tech in found_techs:
                    total_techs += 1
                    name = tech['tile']
                    if name in techs.keys():
                        dat[pos + 'tech_taken_' + techs[name]] = True
                    elif name in adv_techs.keys():
                        dat[pos + 'adv_tech_taken_' + adv_techs[name]] = True
                    else:
                        dat[pos + 'tech_taken_' + adv_techs[name]] = True
                dat[pos + 'total_techs_taken'] = total_techs
                
                #buildings
                dat = parse_tree_builds(dat, tree, pos, faction)
                
                
            dat = parse_tree_moves(dat, tree, num_players)
            dat['average_elo'] = tot_elo / num_players
            dat = pd.DataFrame(dat,index=[0])
            return True, dat, num_players, errors_set
        except:
            errors_set[tree['_id']] = traceback.format_exc()
            return False, "", "", errors_set
            
    else:
        return False, "", "", errors_set

In [169]:
'''
ToDo: parallelize this so its faster
'''
with open("game_data_raw.txt", "r") as game_data_raw:
    
    pbar = tqdm(total=total_lines)
    
    two_players = None
    three_players = None
    four_players = None
    
    errors_set = dict()
    
    lines = game_data_raw.readlines()
    line_num = 0
    for line in lines:
            game_tree = json.loads(line)
            success, df, num_player, errors_set = parse_tree(game_tree, errors_set)
            if success:
                if num_player == 2:
                    if two_players is not None:
                        two_players = pd.concat([two_players, df], axis=0, join="outer", ignore_index=True)
                    else:
                        two_players = df
                elif num_player == 3:
                    if three_players is not None:
                        three_players = pd.concat([three_players, df], axis=0, join="outer", ignore_index=True)
                    else:
                        three_players = df
                else:
                    if four_players is not None:
                        four_players = pd.concat([four_players, df], axis=0, join="outer", ignore_index=True)
                    else:
                        four_players = df
            pbar.update(1)

  0%|          | 0/64156 [00:00<?, ?it/s]

In [170]:
len(errors_set)

9530

In [171]:
# monitor parsing errors
err_types = set(errors_set.values())
err_to_count = {}
for err in err_types:
    count = sum(map(str(err).__eq__, errors_set.values()))
    err_to_count[err] = count

for key in err_to_count.keys():
    print(key)
    print(err_to_count[key])
    print('\n------')

Traceback (most recent call last):
  File "/tmp/ipykernel_1284110/1590915440.py", line 17, in parse_tree
    raise ValueError('this game is with expansions: ' + ''.join(tree['game']['expansions']))
ValueError: this game is with expansions: spaceships

28

------
Traceback (most recent call last):
  File "/tmp/ipykernel_1284110/1590915440.py", line 73, in parse_tree
    pos = "pos_" + str(tree['players'][i]['ranking']) + "_"
KeyError: 'ranking'

332

------
Traceback (most recent call last):
  File "/tmp/ipykernel_1284110/1590915440.py", line 57, in parse_tree
    raise NameError('unknown tech: ' + name)
NameError: unknown tech: tech-ship0

9024

------
Traceback (most recent call last):
  File "/tmp/ipykernel_1284110/1590915440.py", line 17, in parse_tree
    raise ValueError('this game is with expansions: ' + ''.join(tree['game']['expansions']))
ValueError: this game is with expansions: frontiers

32

------
Traceback (most recent call last):
  File "/tmp/ipykernel_1284110/1590915440.

In [172]:
# get examples to check out jsons
# JSON: https://www.boardgamers.space/api/game/ + key
# see if game loads (usually doesnt if error): https://www.boardgamers.space/game/ + key
for err in err_types:
    for key in errors_set.keys():
        if errors_set[key] == err:
            print('Game Name: ' + str(key) + ' Total Error Count: ' + str(err_to_count[err]) + '\n')
            print(err)
            print('\n------')
            break

Game Name: Red-list-2093 Total Error Count: 28

Traceback (most recent call last):
  File "/tmp/ipykernel_1284110/1590915440.py", line 17, in parse_tree
    raise ValueError('this game is with expansions: ' + ''.join(tree['game']['expansions']))
ValueError: this game is with expansions: spaceships


------
Game Name: OTB Total Error Count: 332

Traceback (most recent call last):
  File "/tmp/ipykernel_1284110/1590915440.py", line 73, in parse_tree
    pos = "pos_" + str(tree['players'][i]['ranking']) + "_"
KeyError: 'ranking'


------
Game Name: Pax-Magnifica Total Error Count: 9024

Traceback (most recent call last):
  File "/tmp/ipykernel_1284110/1590915440.py", line 57, in parse_tree
    raise NameError('unknown tech: ' + name)
NameError: unknown tech: tech-ship0


------
Game Name: PegliVSSestri26 Total Error Count: 32

Traceback (most recent call last):
  File "/tmp/ipykernel_1284110/1590915440.py", line 17, in parse_tree
    raise ValueError('this game is with expansions: ' + ''.

In [173]:
two_players.to_pickle("two_players_data")
three_players.to_pickle("three_players_data")
four_players.to_pickle("four_players_data")

# dat_2.to_pickle("two_players_data", protocol=3)
# dat_3.to_pickle("three_players_data", protocol=3)
# dat_4.to_pickle("four_players_data", protocol=3)


In [174]:
two_players.to_csv("two_players_data.csv")
three_players.to_csv("three_players_data.csv")
four_players.to_csv("four_players_data.csv")

In [155]:
four_players['pos_1_score'].describe()

count    10982.000000
mean       175.555090
std         20.185165
min         40.000000
25%        163.000000
50%        175.000000
75%        188.000000
max        251.000000
Name: pos_1_score, dtype: float64

In [156]:
four_players[four_players['pos_1_score'] > 230]

,id,map_layout,balance_varient,num_players,1k_1o,2pwt_1o,1qic_2c,2c_terra,2pw_nav,1o_1vp_per_mine,1o_2vp_per_ts,1k_3vp_per_rl,4pw_4vp_per_3pip,4c_1vp_per_gaia,tech_terra,tech_nav,tech_int,tech_gaia,tech_eco,tech_sci,tech_free1,tech_free2,tech_free3,tech_adv-terra,tech_adv-nav,tech_adv-int,tech_adv-gaia,tech_adv-eco,tech_adv-sci,round_1_scoring,round_2_scoring,round_3_scoring,round_4_scoring,round_5_scoring,round_6_scoring,final_scoring_1,final_scoring_2,pos_2_dropped,pos_2_elo,pos_2_faction,pos_2_score,pos_2_start_pos,pos_2_bid,pos_2_feds_taken,pos_2_fed_12vp,pos_2_fed_qic,pos_2_fed_2pw,pos_2_fed_2o,pos_2_fed_6c,pos_2_fed_2k,...,pos_1_score_qic,pos_1_score_round4,pos_1_score_adv,pos_1_score_bid,pos_1_score_gleens,pos_1_score_gaia,pos_1_score_round5,pos_4_score_fed,pos_4_score_round2,pos_4_score_init,pos_4_score_round1,pos_4_score_booster,pos_4_score_charge,pos_4_score_final1,pos_4_score_final2,pos_4_score_resource,pos_4_score_round3,pos_4_score_round6,pos_4_score_tech,pos_4_score_qic,pos_4_score_round4,pos_4_score_adv,pos_4_score_bid,pos_4_score_gleens,pos_4_score_gaia,pos_4_score_round5,pos_3_score_fed,pos_3_score_round2,pos_3_score_init,pos_3_score_round1,pos_3_score_booster,pos_3_score_charge,pos_3_score_final1,pos_3_score_final2,pos_3_score_resource,pos_3_score_round3,pos_3_score_round6,pos_3_score_tech,pos_3_score_qic,pos_3_score_round4,pos_3_score_adv,pos_3_score_bid,pos_3_score_gleens,pos_3_score_gaia,pos_3_score_round5,average_elo,pos_3_research_level_dip,pos_4_research_level_dip,pos_2_research_level_dip,pos_1_research_level_dip
252,PinkyUncle-Match68,standard,beta,4,True,False,True,True,True,True,False,True,True,False,1k_1c,4pw,7vp,1q_1o,4pip,3vp_per_gaia_place,1o_1pw,k_per_terra,4c,3vp_per_rl_pass,3vp_per_fed_pass,5vp_per_fed,3vp_per_mine_place,3o_action,1qic_5c_action,4vp_per_gaia_place,5vp_per_3pip_place,2vp_per_research_bump,5vp_per_3pip_place,4vp_per_ts_place,3vp_per_ts_place,satellite,structureFed,False,308.0,gleens,189.0,2.0,11.0,4.0,0.0,1.0,1.0,1.0,0.0,0.0,...,26,10,44,-18,0,0,8,38.0,5.0,10.0,4.0,11.0,-19.0,6.0,12.0,0.0,2.0,3.0,7.0,11.0,0.0,0.0,-1.0,0.0,0.0,4.0,31.0,0.0,10.0,8.0,9.0,-21.0,12.0,3.0,2.0,6.0,9.0,16.0,0.0,10.0,0.0,0.0,0.0,9.0,12.0,161.25,NaN,NaN,NaN,NaN
413,kome-0113,standard,standard,4,False,True,True,True,False,True,True,True,True,False,k_per_terra,1q_1o,1o_1pw,4c,3vp_per_gaia_place,1k_1c,4pip,7vp,4pw,3o_action,1vp_per_terra_pass,3vp_per_mine_place,3k_action,5vp_per_fed,2vp_per_tech_bump,4vp_per_gaia_place,5vp_per_3pip_place,4vp_per_ts_place,2vp_per_mine_place,5vp_per_fed_place,2vp_per_terra,planetType,gaia,False,304.0,ambas,189.0,4.0,0.0,5.0,2.0,1.0,1.0,0.0,1.0,0.0,...,23,2,25,0,0,0,15,28.0,5.0,10.0,0.0,11.0,-6.0,9.0,9.0,0.0,8.0,4.0,7.0,0.0,6.0,22.0,0.0,0.0,0.0,10.0,21.0,5.0,10.0,8.0,22.0,-8.0,9.0,0.0,3.0,16.0,10.0,0.0,7.0,4.0,18.0,0.0,0.0,0.0,10.0,285.50,0.0,0.0,0.0,0.0
583,Red-sign-5478,standard,standard,4,True,True,False,True,False,False,True,True,True,True,1q_1o,1o_1pw,k_per_terra,1k_1c,4pip,3vp_per_gaia_place,4pw,4c,7vp,3vp_per_fed_pass,3vp_per_rl_pass,3k_action,5vp_per_fed,1qic_5c_action,4vp_per_ts,2vp_per_research_bump,2vp_per_terra,5vp_per_3pip_place,5vp_per_fed_place,5vp_per_3pip_place,4vp_per_gaia_place,structureFed,planetType,False,276.0,ambas,50.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,21,5,45,0,0,0,5,0.0,0.0,10.0,4.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,4.0,3.0,0.0,0.0,9.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,288.50,NaN,NaN,NaN,NaN
682,Swift-beetle-7855,standard,more-balanced,4,True,True,True,True,False,False,True,False,True,True,4c,k_per_terra,1o_1pw,4pw,1q_1o,4pip,3vp_per_gaia_place,7vp,1k_1c,2vp_per_gaia,3vp_per_mine_place,4vp_per_ts,3vp_per_rl_pass,1qic_5c_action,3o_action,4vp_per_ts_place,2vp_per_research_bump,2vp_per_mine_place,5vp_per_3pip_place,5vp_per_fed_place,2vp_per_terra,structure,satellite,False,30.0,terrans,151.0,2.0,0.0,3.0,0.0,0.0,0.0,2.0,0.0,1.0,...,21,10,8,0,8,0,25,0.0,2.0,10.0,4.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0

In [175]:
four_players['balance_variant'].value_counts()

standard         8088
more-balanced    2656
beta              238
Name: balance_varient, dtype: int64

In [176]:
tech_slots = {
        'terra',
        'nav',
        'int',
        'gaia',
        'eco',
        'sci'
    }

    # keeping rounds and final scorings seperate for better analysis
    # starting to reach mem limits on server
point_types = {
        'init',
        'bid',
        'charge',
        'booster',
        'round1',
        'round2',
        'round3',
        'round4',
        'round5',
        'round6',
        'fed',
        'adv',
        'tech_basic',
        'qic',
        'gaia',
        'gleens',
        'final1',
        'final2',
        'resource'
    }

for i in range(2):
    pos = i + 1
    two_players['pos_' + str(pos) +'_tot_score'] = 0
    prefix = 'pos_' + str(pos) + '_score_'
    for pt in point_types:
        two_players['pos_' + str(pos) +'_tot_score'] += two_players[prefix + pt]
    two_players['pos_' + str(pos) +'_tot_score'] += two_players['pos_' + str(pos) +'_tech_score']


/tmp/ipykernel_1284110/1746155298.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  two_players['pos_' + str(pos) +'_tot_score'] = 0
/tmp/ipykernel_1284110/1746155298.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  two_players['pos_' + str(pos) +'_tot_score'] = 0


In [177]:
two_p = two_players[two_players['pos_1_score'] - two_players['pos_1_tot_score'] != 0]

In [179]:
two_p[[
    'id',
    'pos_1_score', 
    'pos_1_tot_score',
    'pos_1_score_init',
    'pos_1_score_bid',
    'pos_1_score_charge',
    'pos_1_score_booster',
    'pos_1_score_round1',
    'pos_1_score_round2',
    'pos_1_score_round3',
    'pos_1_score_round4',
    'pos_1_score_round5',
    'pos_1_score_round6',
    'pos_1_tech_score',
    'pos_1_score_fed',
    'pos_1_score_adv',
    'pos_1_score_tech_basic',
    'pos_1_score_qic',
    'pos_1_score_gaia',
    'pos_1_score_gleens',
    'pos_1_score_final1',
    'pos_1_score_final2',
    'pos_1_score_resource'
]]

,id,pos_1_score,pos_1_tot_score,pos_1_score_init,pos_1_score_bid,pos_1_score_charge,pos_1_score_booster,pos_1_score_round1,pos_1_score_round2,pos_1_score_round3,pos_1_score_round4,pos_1_score_round5,pos_1_score_round6,pos_1_tech_score,pos_1_score_fed,pos_1_score_adv,pos_1_score_tech_basic,pos_1_score_qic,pos_1_score_gaia,pos_1_score_gleens,pos_1_score_final1,pos_1_score_final2,pos_1_score_resource
